In [12]:
import datetime
import numpy as np
import os
import pandas as pd
import sched
import time
import yfinance as yf

tick_seconds = 30
def current_stock_df_name(stock_name):
    return "current_" + stock_name + ".csv"

def history_stock_df_name(stock_name):
    return "history_" + stock_name + ".csv"

def initialize_file_if_not_exists(filename, file_content):
    if not (os.path.isfile(filename)):
        initialized_file = open(filename, "w")
        initialized_file.write(file_content)
        initialized_file.close()
    
requests_filename = "current_requests.csv"
stats_filename = "requests_stats.csv"
stats_header = "timestamp,stock,requests\n"
stock_df_header = "Datetime,Open,High,Low,Close,Adj Close,Volume\n"

def update_stocks_collect_stat(scheduler): 
    import_interval = "5m"
    period_of_import = "60m"
    cur_df = yf.download(tickers = "TSLA", period = period_of_import, interval = import_interval)
    iteration_timestamp = datetime.datetime.now()
    global tick_seconds, requests_filename, stats_filename, stats_header, stock_df_header
    
    if (os.path.isfile(requests_filename)):
        requests = open(requests_filename, "r")
        requests_heading = requests.readline()
        initialize_file_if_not_exists(stats_filename, stats_header)
        stats = open(stats_filename, "a")
        for raw_row in requests:
            row = raw_row.split(",")
            stock_name = row[0]
            try:
                if (os.path.isfile(current_stock_df_name(stock_name))):
                    initialize_file_if_not_exists(history_stock_df_name(stock_name), stock_df_header)
                    history_df = open(history_stock_df_name(stock_name), "a")
                    
                    cur_df = open(current_stock_df_name(stock_name), "r")
                    cur_df.readline() #Skip heading
                    for cur_df_row in cur_df:
                        history_df.write(cur_df_row)
                    cur_df.close()
                    history_df.close()
                cur_df = yf.download(tickers = stock_name, period = period_of_import, interval = import_interval)
                cur_df.to_csv(current_stock_df_name(stock_name))
            except:
                print(f"Error has occured")
            stats.write(str(iteration_timestamp) + ",")
            stats.write(raw_row)
            os.remove(requests_filename)
        stats.write("\n")
        requests.close()
        stats.close()
    print("Iteration has finished")
    scheduler.enter(tick_seconds, 1, update_stocks_collect_stat, (scheduler,))

my_scheduler = sched.scheduler(time.time, time.sleep)
my_scheduler.enter(tick_seconds, 1, update_stocks_collect_stat, (my_scheduler,))
my_scheduler.run()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


FileNotFoundError: [Errno 2] No such file or directory: 'current_requests.csv'

In [8]:
cur_df = yf.download(tickers = "TSLA", period = "5d", interval = "60m")

[*********************100%%**********************]  1 of 1 completed
